In [1]:
from moccasin import setup_notebook

setup_notebook()

In [2]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(f"Active network: {active_network.name}")

Active network: eth-forked


In [3]:
import boa
from boa.contracts.abi.abi_contract import ABIContract
from moccasin.config import get_active_network, Network

STARTING_ETH_BALANCE = int(1000e18)  # 1000 ETH
STARTING_WETH_BALANCE = int(1e18)  # 1 wETH
STARTING_USDC_BALANCE = int(100e6)  # 100 USDC (6 decimals)


def _add_eth_balance() -> None:
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)


def _add_token_balance(usdc: ABIContract, weth: ABIContract) -> None:
    weth.deposit(value=STARTING_WETH_BALANCE)

    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)


def setup_script() -> (ABIContract, ABIContract, ABIContract, ABIContract):
    print("Starting setup script...")

    # Give ourselves some ETH

    # Give ourselves some USDC and wETH
    active_network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)

    return (usdc, weth)


def moccasin_main():
    (usdc, weth) = setup_script()
    return (usdc, weth)


moccasin_main()

Starting setup script...


(<usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>,
 <weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>)

In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [10]:
from moccasin.config import get_config

config = get_config()
config.reload()
active_network = get_active_network()

In [11]:
aave_pool_address_provider = active_network.manifest_named("aave_pool_address_provider")
pool_address = aave_pool_address_provider.getPool()
print(f"Aave Pool address: {pool_address}")

Aave Pool address: 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [12]:
config.reload()
active_network = get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)

In [ ]:
REFERRAL_CODE = 0

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} of token {token.name} to Aave Pool {pool_contract.address}...")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(
    totalCollateralBase,
    totalDebtBase,
    availableBorrowBase,
    currentLiquidationThreshold,
    ltv,
    healthFactor,
) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
    totalCollateralBase: {totalCollateralBase}
    totalDebtBase: {totalDebtBase}
    availableBorrowBase: {availableBorrowBase}
    currentLiquidationThreshold: {currentLiquidationThreshold}
    ltv: {ltv}
    healthFactor: {healthFactor}
""")